In [38]:
import random

import pandas as pd
import numpy as np
import tensorflow as tf
from keras import backend as K
from keras.optimizers import Adam
from keras.initializers import VarianceScaling
from keras.layers import Input, Dense
from keras.models import Model
from keras.utils import to_categorical
from keras.callbacks import Callback
from sklearn.metrics import roc_auc_score


Using TensorFlow backend.


In [17]:
df = pd.read_csv("./data/lgb.csv")
df

,userid,feedid,date_,device,read_comment,comment,like,play,stay,click_avatar,...,userid_in_feedid_nunique,feedid_in_userid_nunique,userid_in_authorid_nunique,authorid_in_userid_nunique,userid_authorid_count,userid_in_authorid_count_prop,authorid_in_userid_count_prop,videoplayseconds_in_userid_mean,videoplayseconds_in_authorid_mean,feedid_in_authorid_nunique
0,8,71474,1,1,0.0,0.0,1.0,500.0,5366.0,0.0,...,394,260,510,190,2,0.003105,0.007664,12642.308,10255.0550,5
1,8,73916,1,1,0.0,0.0,0.0,250.0,1533.0,0.0,...,1429,260,1429,190,1,0.000683,0.003832,12642.308,16000.0000,1
2,8,50282,1,1,0.0,0.0,0.0,750.0,1302.0,0.0,...,2492,260,2995,190,1,0.000265,0.003832,12642.308,28736.4940,24
3,8,11391,1,1,0.0,0.0,1.0,3750.0,5191.0,0.0,...,299,260,1207,190,3,0.001360,0.011500,12642.308,6875.7370,17
4,8,27349,1,1,0.0,0.0,0.0,250.0,800.0,0.0,...,29,260,277,190,2,0.004080,0.007664,12642.308,17253.5780,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7739862,133812,56450,15,2,NaN,NaN,NaN,NaN,NaN,NaN,...,149,274,237,177,1,0.004080,0.003597,17314.080,14221.3120,3
7739863,231669,76501,15,1,NaN,NaN,NaN,NaN,NaN,NaN,...,90,223,628,161,2,0.002080,0.008930,16950.672,6608.3335,19
7739864,179168,70550,15,2,NaN,NaN,NaN,NaN,NaN,NaN,...,385,744,4083,548,2,0.000263,0.002659,26476.697,37323.2150,86
7739865,92546,49432,15,2,NaN,NaN,NaN,NaN,NaN,NaN,...,811,282,6013,203,4,0.000294,0.013750,52482.758,13586.7250,18


In [18]:
df.columns

Index(['userid', 'feedid', 'date_', 'device', 'read_comment', 'comment',
       'like', 'play', 'stay', 'click_avatar', 'forward', 'follow', 'favorite',
       'authorid', 'videoplayseconds', 'is_finish', 'play_times',
       'userid_5day_count', 'userid_5day_finish_rate',
       'userid_5day_play_times_max', 'userid_5day_play_times_mean',
       'userid_5day_play_max', 'userid_5day_play_mean', 'userid_5day_stay_max',
       'userid_5day_stay_mean', 'userid_5day_read_comment_sum',
       'userid_5day_read_comment_mean', 'userid_5day_like_sum',
       'userid_5day_like_mean', 'userid_5day_click_avatar_sum',
       'userid_5day_click_avatar_mean', 'userid_5day_forward_sum',
       'userid_5day_forward_mean', 'feedid_5day_count',
       'feedid_5day_finish_rate', 'feedid_5day_play_times_max',
       'feedid_5day_play_times_mean', 'feedid_5day_play_max',
       'feedid_5day_play_mean', 'feedid_5day_stay_max',
       'feedid_5day_stay_mean', 'feedid_5day_read_comment_sum',
       'feedid_5d

In [23]:
play_cols = ['is_finish', 'play_times', 'play', 'stay']
y_list = ['read_comment', 'like', 'click_avatar', 'forward', 'favorite', 'comment', 'follow']
max_day = 15

In [19]:
train = df[~df['read_comment'].isna()].reset_index(drop=True)
test = df[df['read_comment'].isna()].reset_index(drop=True)


In [24]:
cols = [f for f in df.columns if f not in ['date_'] + play_cols + y_list]

In [26]:
print(cols)
print(train[cols].shape)

['userid', 'feedid', 'device', 'authorid', 'videoplayseconds', 'userid_5day_count', 'userid_5day_finish_rate', 'userid_5day_play_times_max', 'userid_5day_play_times_mean', 'userid_5day_play_max', 'userid_5day_play_mean', 'userid_5day_stay_max', 'userid_5day_stay_mean', 'userid_5day_read_comment_sum', 'userid_5day_read_comment_mean', 'userid_5day_like_sum', 'userid_5day_like_mean', 'userid_5day_click_avatar_sum', 'userid_5day_click_avatar_mean', 'userid_5day_forward_sum', 'userid_5day_forward_mean', 'feedid_5day_count', 'feedid_5day_finish_rate', 'feedid_5day_play_times_max', 'feedid_5day_play_times_mean', 'feedid_5day_play_max', 'feedid_5day_play_mean', 'feedid_5day_stay_max', 'feedid_5day_stay_mean', 'feedid_5day_read_comment_sum', 'feedid_5day_read_comment_mean', 'feedid_5day_like_sum', 'feedid_5day_like_mean', 'feedid_5day_click_avatar_sum', 'feedid_5day_click_avatar_mean', 'feedid_5day_forward_sum', 'feedid_5day_forward_mean', 'authorid_5day_count', 'authorid_5day_finish_rate', 'au

In [27]:
trn_x = train[train['date_'] < 14].reset_index(drop=True)
val_x = train[train['date_'] == 14].reset_index(drop=True)

In [33]:
print(trn_x.columns)
trn_x

Index(['userid', 'feedid', 'date_', 'device', 'read_comment', 'comment',
       'like', 'play', 'stay', 'click_avatar', 'forward', 'follow', 'favorite',
       'authorid', 'videoplayseconds', 'is_finish', 'play_times',
       'userid_5day_count', 'userid_5day_finish_rate',
       'userid_5day_play_times_max', 'userid_5day_play_times_mean',
       'userid_5day_play_max', 'userid_5day_play_mean', 'userid_5day_stay_max',
       'userid_5day_stay_mean', 'userid_5day_read_comment_sum',
       'userid_5day_read_comment_mean', 'userid_5day_like_sum',
       'userid_5day_like_mean', 'userid_5day_click_avatar_sum',
       'userid_5day_click_avatar_mean', 'userid_5day_forward_sum',
       'userid_5day_forward_mean', 'feedid_5day_count',
       'feedid_5day_finish_rate', 'feedid_5day_play_times_max',
       'feedid_5day_play_times_mean', 'feedid_5day_play_max',
       'feedid_5day_play_mean', 'feedid_5day_stay_max',
       'feedid_5day_stay_mean', 'feedid_5day_read_comment_sum',
       'feedid_5d

,userid,feedid,date_,device,read_comment,comment,like,play,stay,click_avatar,...,userid_in_feedid_nunique,feedid_in_userid_nunique,userid_in_authorid_nunique,authorid_in_userid_nunique,userid_authorid_count,userid_in_authorid_count_prop,authorid_in_userid_count_prop,videoplayseconds_in_userid_mean,videoplayseconds_in_authorid_mean,feedid_in_authorid_nunique
0,8,71474,1,1,0.0,0.0,1.0,500.0,5366.0,0.0,...,394,260,510,190,2,0.003105,0.007664,12642.308,10255.055,5
1,8,73916,1,1,0.0,0.0,0.0,250.0,1533.0,0.0,...,1429,260,1429,190,1,0.000683,0.003832,12642.308,16000.000,1
2,8,50282,1,1,0.0,0.0,0.0,750.0,1302.0,0.0,...,2492,260,2995,190,1,0.000265,0.003832,12642.308,28736.494,24
3,8,11391,1,1,0.0,0.0,1.0,3750.0,5191.0,0.0,...,299,260,1207,190,3,0.001360,0.011500,12642.308,6875.737,17
4,8,27349,1,1,0.0,0.0,0.0,250.0,800.0,0.0,...,29,260,277,190,2,0.004080,0.007664,12642.308,17253.578,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708841,250236,72813,12,2,0.0,0.0,0.0,6369.0,6566.0,0.0,...,1068,186,2812,172,1,0.000169,0.005207,21958.115,20080.752,40
6708842,250236,52019,12,2,0.0,0.0,0.0,13212.0,13708.0,0.0,...,750,186,750,172,1,0.001310,0.005207,21958.115,11000.000,1
6708843,250236,51045,12,2,0.0,0.0,0.0,0.0,32215.0,0.0,...,112,186,348,172,1,0.001767,0.005207,21958.115,30656.637,16
6708844,250236,46881,12,2,0.0,0.0,0.0,0.0,5618.0,0.0,...,989,186,3216,172,1,0.000236,0.005207,21958.115,14078.376,18


In [34]:
train_df = trn_x
other_df = val_x
# First group of tasks according to the paper
label_columns = y_list[:4]

# One-hot encoding categorical columns
categorical_columns = ['userid', 'feedid', 'authorid']
train_raw_labels = train_df[label_columns]
other_raw_labels = other_df[label_columns]

In [35]:
train_df.drop(label_columns, axis=1)

,userid,feedid,date_,device,comment,play,stay,follow,favorite,authorid,...,userid_in_feedid_nunique,feedid_in_userid_nunique,userid_in_authorid_nunique,authorid_in_userid_nunique,userid_authorid_count,userid_in_authorid_count_prop,authorid_in_userid_count_prop,videoplayseconds_in_userid_mean,videoplayseconds_in_authorid_mean,feedid_in_authorid_nunique
0,8,71474,1,1,0.0,500.0,5366.0,0.0,0.0,1528,...,394,260,510,190,2,0.003105,0.007664,12642.308,10255.055,5
1,8,73916,1,1,0.0,250.0,1533.0,0.0,0.0,1442,...,1429,260,1429,190,1,0.000683,0.003832,12642.308,16000.000,1
2,8,50282,1,1,0.0,750.0,1302.0,0.0,0.0,8648,...,2492,260,2995,190,1,0.000265,0.003832,12642.308,28736.494,24
3,8,11391,1,1,0.0,3750.0,5191.0,0.0,0.0,11976,...,299,260,1207,190,3,0.001360,0.011500,12642.308,6875.737,17
4,8,27349,1,1,0.0,250.0,800.0,0.0,0.0,4370,...,29,260,277,190,2,0.004080,0.007664,12642.308,17253.578,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708841,250236,72813,12,2,0.0,6369.0,6566.0,0.0,0.0,2878,...,1068,186,2812,172,1,0.000169,0.005207,21958.115,20080.752,40
6708842,250236,52019,12,2,0.0,13212.0,13708.0,0.0,0.0,6786,...,750,186,750,172,1,0.001310,0.005207,21958.115,11000.000,1
6708843,250236,51045,12,2,0.0,0.0,32215.0,0.0,0.0,14730,...,112,186,348,172,1,0.001767,0.005207,21958.115,30656.637,16
6708844,250236,46881,12,2,0.0,0.0,5618.0,0.0,0.0,13164,...,989,186,3216,172,1,0.000236,0.005207,21958.115,14078.376,18


In [42]:
# transformed_train = pd.get_dummies(train_df.drop(label_columns, axis=1), columns=categorical_columns)
# transformed_other = pd.get_dummies(other_df.drop(label_columns, axis=1), columns=categorical_columns)
to_categorical((train_raw_labels.read_comment == 1.0).astype(int), num_classes=2)

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [ ]:

# Filling the missing column in the other set
# transformed_other['det_hh_fam_stat_ Grandchild <18 ever marr not in subfamily'] = 0

# One-hot encoding categorical labels
train_income = to_categorical((train_raw_labels.income_50k == ' 50000+.').astype(int), num_classes=2)
train_marital = to_categorical((train_raw_labels.marital_stat == ' Never married').astype(int), num_classes=2)
other_income = to_categorical((other_raw_labels.income_50k == ' 50000+.').astype(int), num_classes=2)
other_marital = to_categorical((other_raw_labels.marital_stat == ' Never married').astype(int), num_classes=2)

dict_outputs = {
    'income': train_income.shape[1],
    'marital': train_marital.shape[1]
}
dict_train_labels = {
    'income': train_income,
    'marital': train_marital
}
dict_other_labels = {
    'income': other_income,
    'marital': other_marital
}
output_info = [(dict_outputs[key], key) for key in sorted(dict_outputs.keys())]

# Split the other dataset into 1:1 validation to test according to the paper
validation_indices = transformed_other.sample(frac=0.5, replace=False, random_state=SEED).index
test_indices = list(set(transformed_other.index) - set(validation_indices))
validation_data = transformed_other.iloc[validation_indices]
validation_label = [dict_other_labels[key][validation_indices] for key in sorted(dict_other_labels.keys())]
test_data = transformed_other.iloc[test_indices]
test_label = [dict_other_labels[key][test_indices] for key in sorted(dict_other_labels.keys())]
train_data = transformed_train
train_label = [dict_train_labels[key] for key in sorted(dict_train_labels.keys())]